# 03 - Model Training

Fine-tune Qwen 2.5 14B using QLoRA with Unsloth.

Two models:
- **Alpha**: Trained on golden traces only (358 examples, 3 epochs)
- **Beta**: Trained on golden + game traces (508 examples, 2.5 epochs)

In [ ]:
# Install dependencies (run once)
# !pip install unsloth transformers datasets trl peft

In [ ]:
import torch
import json
from pathlib import Path
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset

## Configuration

In [ ]:
# Model config
BASE_MODEL = "unsloth/Qwen2.5-14B-Instruct"
MAX_SEQ_LENGTH = 2048

# LoRA config
LORA_RANK = 32  # Higher rank helped for this task
LORA_ALPHA = 64
LORA_DROPOUT = 0.05

# Training config
LEARNING_RATE = 2e-4
BATCH_SIZE = 2
GRAD_ACCUM = 4  # Effective batch size = 8

# Choose which model to train
TRAIN_ALPHA = False  # Set True to train Alpha, False for Beta

if TRAIN_ALPHA:
    DATA_PATH = "data/sft_alpha.jsonl"
    EPOCHS = 3
    SAVE_NAME = "qwen-connections-alpha-r32-ep3"
else:
    DATA_PATH = "data/sft_beta.jsonl"
    EPOCHS = 2.5  # 2.5 epochs worked better than 3 for beta
    SAVE_NAME = "qwen-connections-beta-r32-ep2.5"

OUTPUT_DIR = f"outputs_{SAVE_NAME}"

## Load Base Model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print(f"Loaded {BASE_MODEL}")

## Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,} / {total:,} ({trainable/total*100:.2f}%)")

## Load Training Data

In [ ]:
# Load training data
data = []
with open(DATA_PATH, "r") as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))

print(f"Loaded {len(data)} training examples from {DATA_PATH}")

# Count sources
sources = {}
for ex in data:
    src = ex.get("metadata", {}).get("source", "unknown")
    sources[src] = sources.get(src, 0) + 1
print(f"Sources: {sources}")

In [ ]:
# Create dataset
dataset = Dataset.from_list(data)

# Format using chat template
def format_chat(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_chat)

# Check a sample
print(f"\nSample (first 500 chars):")
print(dataset[0]["text"][:500])

## Train

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        warmup_steps=10,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        output_dir=OUTPUT_DIR,
        optim="adamw_8bit",
        seed=42,
        save_strategy="epoch",
    ),
)

print(f"Training {SAVE_NAME}...")
print(f"  Data: {len(data)} examples")
print(f"  Epochs: {EPOCHS}")
print(f"  Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")
print()

In [ ]:
# Train!
trainer.train()
print("\nTraining complete!")

## Save Model

In [ ]:
# Save LoRA weights
model.save_pretrained(SAVE_NAME)
tokenizer.save_pretrained(SAVE_NAME)
print(f"Model saved to {SAVE_NAME}")

# Also backup to /data for persistence (if using RunPod)
import shutil
backup_path = f"/data/{SAVE_NAME}"
shutil.copytree(SAVE_NAME, backup_path, dirs_exist_ok=True)
print(f"Backed up to {backup_path}")

## Quick Test

In [ ]:
# Test the trained model
FastLanguageModel.for_inference(model)

test_prompt = """REMAINING: CAT, DOG, FISH, BIRD, RED, BLUE, GREEN, YELLOW, ONE, TWO, THREE, FOUR, APPLE, BANANA, ORANGE, GRAPE

Find 4 words that share a hidden theme.

Think step by step:
1. What patterns do you see?
2. Which 4-word group are you MOST confident about?
3. Verify: Are all 4 words in the REMAINING list?

Output your most confident group:
{"group": ["W1", "W2", "W3", "W4"]}"""

messages = [
    {"role": "system", "content": "NYT Connections. Find groups of 4. Output JSON."},
    {"role": "user", "content": test_prompt}
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.3, do_sample=True)

response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print("Model response:")
print(response)

## Training Loss Reference

Expected loss curves:

**Alpha (358 examples, 3 epochs):**
- Start: ~1.2-1.3
- End: ~0.35-0.40

**Beta (508 examples, 2.5 epochs):**
- Start: ~1.2-1.3  
- End: ~0.35-0.40

If loss drops below 0.2, you may be overfitting. If starting loss is ~0.4, you accidentally loaded an already fine-tuned model!